In [1]:
import os
import glob
import time

In [2]:
import numpy as np

In [3]:
from keras import applications
from keras.applications.vgg19 import preprocess_input
from keras.preprocessing import image
from keras import applications
from keras.models import Model

Using TensorFlow backend.


In [4]:
base_model = applications.vgg19.VGG19(include_top=True, weights='imagenet', input_tensor=None, 
                                      input_shape=None, pooling=None, classes=1000)


574717952/574710816 [==============================] - 282s 0us/step


In [7]:
data_path = os.path.abspath('../../data/umpire_non_umpire_train')


In [8]:
save_path = os.path.abspath('../../features/vgg19')

In [9]:
classes = ['non_umpire', 'umpire']

In [10]:
start_time = time.time()
layers_to_extract = ['fc1']

for layer_num in range(0, len(layers_to_extract)):
    model = Model(input=base_model.input, output=base_model.get_layer(layers_to_extract[layer_num]).output)
    for cls in range(1, 3):
        img_count = 0
        feats=[]
        for image_path in glob.glob(f'{data_path}/{classes[cls-1]}*'):
            img_count=img_count+1
            print(img_count)
        
            #Pre-processing
            img = image.load_img(image_path, target_size=(224, 224))
            x_in = image.img_to_array(img)
            x_in = np.expand_dims(x_in, axis=0)
            x_in = preprocess_input(x_in)
            
            #Feature Extraction
            features = model.predict(x_in)
            features = features.flatten()
            feats.append(features)
            features_arr = np.char.mod('%f', features)
        
        feature_list = np.squeeze(np.asarray(feats))
        labels = np.ones(len(feature_list))*cls
        feature_list = np.column_stack((feature_list, labels))
        #Save the features as numpy array for further processing
        np.save(f'{save_path}/model1_vgg19_{layers_to_extract[layer_num]}_features_{classes[cls-1]}.npy',
                feature_list)

print("--- %s seconds ---" % (time.time() - start_time))

--- 0.0 seconds ---


c:\users\tanmay jain\appdata\local\programs\python\python37\lib\site-packages\ipykernel_launcher.py:5: UserWarning: Update your `Model` call to the Keras 2 API: `Model(inputs=Tensor("in..., outputs=Tensor("fc...)`
  """
